
#### Run the cell below to install the required packages for Copilot


In [1]:

#Run this cell to install the required packages for Copilot
%load_ext dscopilot_installer
%activate_dscopilot


StatementMeta(, 8e1cceac-f7d8-4631-815c-1635fd001655, 8, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.8/496.8 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 94.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 183.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 178.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.9/220.9 kB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 113.0 

### **Chat_magic commands**

* Using installed DSCopilot package.
* Service: fabric
* Model: gpt-35-turbo-0125




#### **AI Processor Disclaimer**


#### Data Privacy and Security

Chat-magics is powered by Azure OpenAI Service and is subject to the supplemental terms of use for
[Microsoft Azure Previews](https://azure.microsoft.com/en-us/support/legal/preview-supplemental-terms/).
Azure OpenAI is fully controlled by Microsoft.

In order to generate a response, chat-magics uses (a) your prompt or input and, when appropriate,
(b) additional data that is retrieved through the grounding process to provide more relevant,
contextual responses. This information is sent to Azure OpenAI, where it is processed and an
output is generated. Therefore, data processed by Azure OpenAI can include:

- Your prompt or input
- Grounding data
- Chat-magics response or output

Your Data:

- Is not used to train models.
- Is not available to other customers.
- Is not stored.





### **Usage**
Use the `%chat_magics` command to display this help message.

<details>
<summary>Expand for details...</summary>

To get started, try something like the following:

    %%code
    Load my_data.csv from the current folder into a pandas dataframe.

#### Main Commands
* `%%chat` - Ask questions about your notebook state or let the chat-magics help you understand or author it.
* `%%code` - Generate code to work with or visualize your data.
* `%describe` - Describe a loaded dataframe.
* `%%add_comments` - Add comments to the code in a cell.
* `%%fix_errors` - Fix errors in a cell.
* `%%translate` - Translate code from one language to another.

#### Configuration Commands
* `%set_output` - Set whether code responses are generated to the current cell, next cell, cell output, a variable, or not at all.
* `%set_language` - Set the default language for generated code.

#### Context Commands
* `%pin & %unpin` - Pin and unpin dataframes to better focus AI responses.
* `%new_task` - Provide overall guidance for generated responses. Clears history.
* `%ignore` - Ignore a cell so that it is not processed by chat-magics.

#### Detailed Usage
For detailed information on each command, use the `?` (help operator), e.g. `%%code?` or `%set_sharing_level?`. Also, note that some magic commands may not be available depending on the AI service you have configured.

</details>

In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!

IS_CUSTOM_DATA = False  # if True, dataset has to be uploaded manually

if not IS_CUSTOM_DATA:
    # Download data files into lakehouse if it does not exist
    import os, requests

    remote_url = "https://synapseaisolutionsa.blob.core.windows.net/public/hotel_reviews"
    file_list = ["hotel_reviews_demo.csv"]
    download_path = f"/lakehouse/default/Files/hotel_reviews/raw/"

    if not os.path.exists("/lakehouse/default"):
        raise FileNotFoundError(
            "Default lakehouse not found, please add a lakehouse and restart the session."
        )
    os.makedirs(download_path, exist_ok=True)
    for fname in file_list:
        if not os.path.exists(f"{download_path}/{fname}"):
            r = requests.get(f"{remote_url}/{fname}", timeout=30)
            with open(f"{download_path}/{fname}", "wb") as f:
                f.write(r.content)
    print("Downloaded demo data files into lakehouse.")


StatementMeta(, fb93d0e7-8766-444c-a514-755a33ec3099, 3, Finished, Available, Finished)

Downloaded demo data files into lakehouse.


In [2]:
import synapse.ml.core
from synapse.ml.services import *
from pyspark.sql.functions import col, flatten, udf, lower, trim
from pyspark.sql.types import StringType
from synapse.ml.services.language import AnalyzeText

StatementMeta(, fb93d0e7-8766-444c-a514-755a33ec3099, 4, Finished, Available, Finished)

In [3]:
df1 = spark.read.format("csv").\
     option("header","true").\
     load("Files/hotel_reviews/raw/hotel_reviews_demo.csv")
display(df1)

StatementMeta(, fb93d0e7-8766-444c-a514-755a33ec3099, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7995f2e4-7ee3-4eb8-b998-bdb4ed333d37)

In [4]:
translate = (Translate()
    .setTextCol("reviews_text")
    .setToLanguage("en")
    .setOutputCol("translation")
    .setConcurrency(5))

df_en = translate.transform(df1)\
        .withColumn("translation_result", flatten(col("translation.translations")))\
        .withColumn("translation", col("translation_result.text")[0])\
        .cache()

df_en = df_en.select(df_en.columns[:6]+ ["translation"])

display(df_en)

StatementMeta(, fb93d0e7-8766-444c-a514-755a33ec3099, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, dce9a125-ea9a-4042-adcb-dfa856ea7426)

In [32]:
process_column = udf(lambda x: f"You are a text classifier. Classify the following hotel reviews into 1 of the following categories: categories: [Service, Location, Facilities, Sanitation]. reviews_text:{x},Classified categor: ", StringType())
df_en_key_prompt = df_en.withColumn("prompt", process_column(df_en["translation"])).cache()
display(df_en_key_prompt.tail(5))

StatementMeta(, fb93d0e7-8766-444c-a514-755a33ec3099, 44, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 002677b6-e79f-4e4c-b718-7b79fdeb4322)

In [33]:
deployment_name = "gpt-4o"
completion = (
    OpenAICompletion()
    .setDeploymentName(deployment_name)
    .setMaxTokens(200)
    .setPromptCol("prompt")
    .setErrorCol("error")
    .setOutputCol("classification")
)
completed_df = completion.transform(df_en_key_prompt)\
                .withColumn("class", trim(lower(col("classification.choices.text")[0])))\
                .cache()

df_final = completed_df.select(completed_df.columns[:8]+["class"])
display(df_final.tail(5))

StatementMeta(, fb93d0e7-8766-444c-a514-755a33ec3099, 45, Submitted, Running, Running)

SynapseWidget(Synapse.DataFrame, 88903eb9-dd7a-4c13-960b-10bb9b452fcd)

In [15]:
delta_table_path = "Tables/hotel_review" #fill in your delta table path
df_final.write.format("delta").mode("overwrite").option('overwriteSchema', 'true').save(delta_table_path)

StatementMeta(, fb93d0e7-8766-444c-a514-755a33ec3099, 17, Finished, Available, Finished)